# 2017年11月13日17:20:39
# 训练CUMT论文提出的结构

In [1]:
import tensorflow as tf
import numpy as np
from cumt_eye import Cumt_itraker
import matplotlib.pyplot as plt
import os
from scipy.misc import imread
from scipy.misc import imresize
import UI
import cv2
import pickle
sess=tf.InteractiveSession()
cumt_data=np.load(r'D:\Proj_DL\Code\Proj_EyeTraker\cumt_data.pkl')
log_name=r'cumtdata256/cumt_ori'

X=tf.placeholder(shape=[None,32,128,3],dtype=tf.float32)
Y=tf.placeholder(shape=[None,10],dtype=tf.float32)
DROPOUT=tf.placeholder(dtype=tf.float32)
LR=tf.placeholder(dtype=tf.float32)
BN_FLAG=tf.placeholder(dtype=tf.bool)

with tf.name_scope('model'):
    model=Cumt_itraker(image=X,bn_flag=BN_FLAG,dropout_rate=DROPOUT)
    y_score=model.score

with tf.name_scope('LOSS'):
    LOSS=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=y_score,labels=Y))
    tf.summary.scalar('loss',LOSS)
with tf.name_scope('TRAIN'):
    TRAIN=tf.train.AdamOptimizer(LR).minimize(LOSS)
with tf.name_scope('ACCURACY'):
    acc_count=tf.equal(tf.arg_max(y_score,1),tf.arg_max(Y,1))
    ACCURACY=tf.reduce_mean(tf.cast(acc_count,tf.float32))
    tf.summary.scalar('acc',ACCURACY)

writer_tr=tf.summary.FileWriter(r'./mylog/'+log_name+r'/train',)
writer_te=tf.summary.FileWriter(r'./mylog/'+log_name+r'/test',)
writer_val=tf.summary.FileWriter(r'./mylog/'+log_name+r'/val',)
merge=tf.summary.merge_all()
ex_ops=tf.get_collection(tf.GraphKeys.UPDATE_OPS)



Cumt_itraker Init Ready!


In [2]:
#初始化变量，需要重新训练才运行
init=tf.global_variables_initializer()
sess.run(init)

In [ ]:
#从保存的检查点钟复原变量，加载原数据训练好的模型
saver=tf.train.Saver()
saver.restore(sess,r'D:\Proj_DL\Code\Proj_EyeTraker\model_save\model_71.ckpt-64600')


In [3]:
cumt_picmean=[103.939, 116.779, 123.68]
d=np.load('valimg1k_1109.pkl')
val={}
val_data=d['data'].astype('float32')-cumt_picmean
val_label=d['label']
del d
print(val_data.shape,val_label.shape)


(966, 32, 128, 3) (966, 10)


In [4]:
data=cumt_data['data']
label=cumt_data['labels']
index=np.arange(data.shape[0])
np.random.shuffle(index)
# tr_index=index[:int(data.shape[0]*0.9)]
# te_index=index[int(data.shape[0]*0.9):]
tr_index=index[:256]
te_index=index[256:]
tr_data,tr_label=data[tr_index],label[tr_index]
te_data,te_label=data[te_index],label[te_index]
del data
del label
del cumt_data
print(tr_data.shape,te_data.shape)


(256, 32, 128, 3) (74573, 32, 128, 3)


In [6]:
%%timeit
mask=np.random.choice(te_data.shape[0],1,replace=False)
x_,y_=te_data[mask],te_label[mask]
feed_dict={X:x_,Y:y_,DROPOUT:1.,BN_FLAG:False}
sess.run(y_score,feed_dict=feed_dict)

19.3 ms ± 10.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [5]:
# 150 loop/min
lr_=1e-3
batch_size=104
acc_his=[]
for i in range(1,1000*5000):
    mask=np.random.choice(tr_data.shape[0],batch_size,replace=False)
    x_,y_=tr_data[mask],tr_label[mask]
    feed_dict={X:x_,Y:y_,DROPOUT:0.5,BN_FLAG:True,LR:lr_}
    sess.run([TRAIN,ex_ops],feed_dict=feed_dict)
    if i%10==0:
        feed_dict={X:x_,Y:y_,DROPOUT:1.,BN_FLAG:False,LR:lr_}
        loss_,acc_,m_=sess.run([LOSS,ACCURACY,merge],feed_dict=feed_dict)
        writer_tr.add_summary(m_,i)
        print('train epoch:{},loss:{:.5f},accuracy:{:.5f}'.format(i,loss_,acc_))
    if i%20==0:
        mask=np.random.choice(te_data.shape[0],128,replace=False)
        x_,y_=te_data[mask],te_label[mask]
        feed_dict={X:x_,Y:y_,DROPOUT:1.,BN_FLAG:False,LR:lr_}
        loss_,acc_,m_=sess.run([LOSS,ACCURACY,merge],feed_dict=feed_dict)
        writer_te.add_summary(m_,i)
        acc_his.append(acc_)
        mean_acc=np.mean(acc_his[-10:])
        print('--test epoch:{},loss:{:.5f},accuracy:{:.5f},mean acc:{:.5f}'.format(i,loss_,acc_,mean_acc))
    if i%30==0:
        mask=np.random.choice(val_data.shape[0],128,replace=False)
        x_,y_=val_data[mask],val_label[mask]
        feed_dict={X:x_,Y:y_,DROPOUT:1.,BN_FLAG:False,LR:lr_}
        loss_,acc_,m_=sess.run([LOSS,ACCURACY,merge],feed_dict=feed_dict)
        writer_val.add_summary(m_,i)
        print('@@val epoch:{},loss:{:.5f},accuracy:{:.5f}'.format(i,loss_,acc_))
    if i%400==0:
        lr_=max(lr_*0.5,1e-5)
        print('$$$ lr change:{}'.format(lr_))

train epoch:10,loss:43.91220,accuracy:0.15385
train epoch:20,loss:12.58156,accuracy:0.23077
--test epoch:20,loss:14.77892,accuracy:0.10156,mean acc:0.10156
train epoch:30,loss:8.47065,accuracy:0.21154
@@val epoch:30,loss:9.92523,accuracy:0.07031
train epoch:40,loss:6.88839,accuracy:0.23077
--test epoch:40,loss:8.86342,accuracy:0.13281,mean acc:0.11719
train epoch:50,loss:7.23467,accuracy:0.22115
train epoch:60,loss:6.57110,accuracy:0.25000
--test epoch:60,loss:8.10667,accuracy:0.19531,mean acc:0.14323
@@val epoch:60,loss:4.81978,accuracy:0.13281
train epoch:70,loss:4.74538,accuracy:0.31731
train epoch:80,loss:5.58902,accuracy:0.29808
--test epoch:80,loss:6.90737,accuracy:0.19531,mean acc:0.15625
train epoch:90,loss:3.73674,accuracy:0.27885
@@val epoch:90,loss:2.94202,accuracy:0.13281
train epoch:100,loss:2.73220,accuracy:0.33654
--test epoch:100,loss:3.87830,accuracy:0.19531,mean acc:0.16406
train epoch:110,loss:2.00008,accuracy:0.42308
train epoch:120,loss:1.06665,accuracy:0.61538
--t

train epoch:890,loss:0.00004,accuracy:1.00000
train epoch:900,loss:0.00004,accuracy:1.00000
--test epoch:900,loss:3.53266,accuracy:0.45312,mean acc:0.49062
@@val epoch:900,loss:3.43525,accuracy:0.38281
train epoch:910,loss:0.00004,accuracy:1.00000
train epoch:920,loss:0.00001,accuracy:1.00000
--test epoch:920,loss:2.73445,accuracy:0.47656,mean acc:0.48516
train epoch:930,loss:0.00002,accuracy:1.00000
@@val epoch:930,loss:4.13096,accuracy:0.35938
train epoch:940,loss:0.00002,accuracy:1.00000
--test epoch:940,loss:2.70606,accuracy:0.56250,mean acc:0.48594
train epoch:950,loss:0.00003,accuracy:1.00000
train epoch:960,loss:0.00002,accuracy:1.00000
--test epoch:960,loss:3.85908,accuracy:0.36719,mean acc:0.47656
@@val epoch:960,loss:4.73398,accuracy:0.33594
train epoch:970,loss:0.00004,accuracy:1.00000
train epoch:980,loss:0.00004,accuracy:1.00000
--test epoch:980,loss:2.84375,accuracy:0.45312,mean acc:0.47188
train epoch:990,loss:0.00003,accuracy:1.00000
@@val epoch:990,loss:3.47362,accurac

KeyboardInterrupt: 

In [ ]:
def count_trainable_params():
    total_parameters = 0
    for variable in tf.trainable_variables():
        shape = variable.get_shape()
        variable_parametes = 1
        for dim in shape:
            variable_parametes *= dim.value
        total_parameters += variable_parametes
    print("Total training params: %.5f Million,%.5f Mb" % (total_parameters / 1e6,total_parameters* 4/ (1024*1024)))
count_trainable_params()

#### 提取所有可训练参数，并保存

In [ ]:
filename='test_cumt.pkl'
all_var_name=list(tf.trainable_variables())
weight_dict={}
#提取变量值
for name_ in all_var_name:
    #变量名称
    layer_name=str(name_).split("'")[1][:-2]
    #print(layer_name)
    with tf.variable_scope('',reuse=True):
        var=tf.get_variable(layer_name)
        #注意var是tensor，需要转换一下
        weight_dict[layer_name]=sess.run(var)
#保存到pkl文件中
fp=open(filename,'wb')
pickle.dump(obj=weight_dict,file=fp)
fp.close()
print('save weight file done!')

#### 加载保存的网络参数

In [ ]:
weight_addr='test_cumt.pkl'
skip_layer=[]
network_dict=np.load(weight_addr)
layer_name=list(network_dict.keys())
for name_ in layer_name:
    if name_ in skip_layer:
        print('skip layer:{}'.format(name_))
        continue
    with tf.variable_scope('',reuse=True):
        var=tf.get_variable(name_)
        sess.run(var.assign(network_dict[name_]))
print('network init done!')

#### 计算所有可训练变量数

In [ ]:

sum_=0
for s in list(tf.trainable_variables()):
    s=str(s)
    shape_=s[s.index('(')+1:s.index(')')].split(',')
    s_=1
    #print(shape_)
    for i in shape_:
        if i =='':continue
        #print(i)
        s_*=int(i)
    sum_+=s_
print('trainable variables nums:{},{}Mb'.format(sum_,sum_*4/(1024*1024)))

In [ ]:
#保存模型
saver=tf.train.Saver()
saver.save(sess,'./model_save/model.ckpt',global_step=1000)

In [ ]:
'''
在视频中插入判别网络
'''

In [ ]:
prob=tf.nn.softmax(y_score,name='prob')
y_guess=tf.arg_max(prob,1)+1

#预测方向到九宫格方向,
pre2block={2:3,3:2,4:1,
           5:6,6:5,7:4,
           8:9,9:8,10:7}
#校正时需要保存图像均值
VGG_MEAN =mean_pic

In [ ]:
#BGR
#VGG_MEAN = [103.939, 116.779, 123.68]


In [ ]:
cap = cv2.VideoCapture(r'test_video/101603.mp4')
fps=cap.get(cv2.CAP_PROP_FPS)
fsize=(
    int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)),
    int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
      )
# save video
video_saver=cv2.VideoWriter('demo_1041.flv',cv2.VideoWriter_fourcc('F','L','V','1'),50,fsize)
while (True):
    ret,frame = cap.read()
    
    if ret==True:
        #截取眼部图片
#         frame=np.transpose(frame,[1,0,2])
#         frame=frame[::-1,:,:]
#         eye_img=None
        #frame=frame[:,::-1,:]
        eye_img=UI.drew_face_eye(frame)
        if eye_img is None:
            #没有检测到眼睛，跳到下一帧
            cv2.imshow('frame',frame)
            video_saver.write(frame)
            if cv2.waitKey(10)& 0xff ==ord('q'):
                print('quite')
                break
            continue
        # BGR -> RGB
        eye_img=eye_img[:,:,::-1]
        e1=imresize(eye_img,(32,128,3)).astype('float32')-VGG_MEAN
        
        x_=e1.reshape((1,32,128,3))
        pre_asw=sess.run(y_guess,feed_dict={X:x_,DROPOUT:1.,BN_FLAG:False})[0]
        #给图片划线，划框
        print(pre_asw)
        frame=UI.drawline(frame,line_w=1)
        if pre_asw != 1:
            #有检测结果
            
            frame=UI.drawblock(frame,pre2block[pre_asw])
            #cv2.waitKey()
        #cv2.imshow('frame',x_.reshape((32,128,3)))
        cv2.imshow('frame',frame)
        video_saver.write(frame)
        if cv2.waitKey(10)& 0xff ==ord('q'):
            print('quite')
            break
    else:
        print('video end')
        break
cap.release()
video_saver.release()
cv2.destroyAllWindows()

In [ ]:
/**************************************************************************/
'''
用固定的眼部识别参数截取眼部图片
标注，保存
'''

In [ ]:
import cv2
import UI
import matplotlib.pyplot as plt
import os
import numpy as np
from scipy.misc import imread
from scipy.misc import imresize

#自然方向 -> 论文方向
key_map={
         1:'10',2:'9',3:'8',
         4:'7',5:'6',6:'5',
         7:'4',8:'3',9:'2',
         0:'1'
        }

In [ ]:
#改
cap = cv2.VideoCapture(r'test_video/101603.mp4')
i_counter=0
while (True):
    i_counter+=1
    ret,frame = cap.read()
    if ret==True:
        #截取眼部图片
        eye_img=UI.drew_face_eye(frame)
        if eye_img is not None:
            cv2.imshow('frame',eye_img)
            a=(cv2.waitKey())-48
            cv2.destroyWindow('frame')
            
            # Esc 退出，'.'表示放弃这张图
            if a==(-21):
                break
            elif a!=-2:
                resize_img=imresize(eye_img,(32,128,3)).astype('uint8')
                cv2.imwrite('./test_video/101603'+str(i_counter)+'_'+key_map[a]+'.jpg',resize_img)
    else:
        print('video end')
        break
cap.release()
cv2.destroyAllWindows()

In [ ]:
img_root='./test_video/101603'
lis=os.listdir(img_root)
img_arr,lab_arr=[],[]
for addr in lis:
    #标签
    lable_=[0]*10
    if addr[-5]==0:
        n=10
    else:
        n=int(addr[-5])
    lable_[n-1]=1
    lab_arr.append(lable_)
    #图像
    img_=imread(os.path.join(img_root,addr))
    #img_=imresize(img_,(32,128,3)).astype('uint8')
    img_arr.append(img_)
img_mat=np.array(img_arr)
lab_mat=np.array(lab_arr)

In [ ]:
img_mat.shape

In [ ]:
import pickle
fp=open('./test_video/101603/101603_z.pkl','wb')
pickle.dump(file=fp,obj={'images':img_mat,'labels':lab_mat})
fp.close()